# Architektur Neuronales Netz

In [1]:
# aNN Architektur

# Importe / Bibliotheken
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path


#### Default Datentyp

In [2]:
torch.set_default_dtype(torch.float64)

#### Erzeugnung des Moduls

In [3]:
class NeuralNetwork(nn.Module):
    
    #Initalisierung der Netzwerk layers
    def __init__(self, input_size, hidden1_size, hidden2_size, output_size):
    
        super().__init__() #Referenz zur Base Class (nn.Module)
        #Kaskade der Layer
        self.linear_afunc_stack = nn.Sequential(
            nn.Linear(input_size, hidden1_size), #Lineare Transformation mit gespeicherten weights und biases
            nn.Tanh(), #Nicht lineare Aktivierungsfunktion um komplexe nichtlineare Zusammenhänge abzubilden
            nn.Linear(hidden1_size, hidden2_size),
            nn.Tanh(),
            nn.Linear(hidden2_size, output_size),
        )

    #Implementierung der Operationen auf Input Daten
    def forward(self, x):
        #x = self.flatten(x)
        out = self.linear_afunc_stack(x)
        return out

#### Ausgabe Modul

In [4]:
model = NeuralNetwork(5, 20, 20, 1)
print(model)

NeuralNetwork(
  (linear_afunc_stack): Sequential(
    (0): Linear(in_features=5, out_features=20, bias=True)
    (1): Tanh()
    (2): Linear(in_features=20, out_features=20, bias=True)
    (3): Tanh()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)


#### DataLoader

In [5]:
batch_size = 32 #Zahl der Datenpaare die vor einem erneuten Update der Parameter ins Netzt gegeben werden
eq_data_file = Path.cwd() / 'data' / 'eq_dataset.npz' #Import der GGW Daten

res = np.load(eq_data_file)

# Bei Speicherung wurden Daten als T, p, x_0 und xi gespeichert
# Inputs T, p, x_0[H2,N2,NH3]
# Outputs xi
# Umwandlen der np.arrays in torch.tensors zur besseren Arbeit mit PyTorch
T = torch.tensor(res['T'])
p = torch.tensor(res['p'])
x_0 = torch.tensor(res['x_0'])
xi = torch.tensor(res['xi'])

# T = torch.tensor(res['T']).float()
# p = torch.tensor(res['p']).float()
# x_0 = torch.tensor(res['x_0']).float()
# xi = torch.tensor(res['xi']).float()

# print(T.dtype)
# print(xi.dtype)

x_input = torch.stack((T,p,x_0[:,0],x_0[:,1],x_0[:,2]),1)
y_output = xi
#print(x_input.size())
# print(xi.size())

# Tensoren zu einem großen Set gruppieren
dataset = TensorDataset(x_input, y_output)

# for x,y in dataset:
#     print(x)
#     print(y)
    
# Split in Trainings und Test Set
train_dataset, test_dataset = random_split(dataset, 
                                           [int(0.8*len(dataset)), int(0.2*len(dataset))], # splitting 80/20
                                           generator = torch.Generator().manual_seed(42)) # Festlegung seed zur Reproduktivität

# Erzeugen der DataLoader zur Arbeit mit Daten
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True) # shuffle batches zur Reduzierung von overfitting
test_dataloader = DataLoader(test_dataset, batch_size = batch_size, shuffle=False)


#### Generierung Netzwerk, Festlegung von loss Funktion und Optimizer

In [6]:
# Erzeugung aNN
net = NeuralNetwork(5, 20, 20, 1)

# Loss Funktion; gibt Fehler an
#loss_fn = nn.MSELoss()

#Definition custom loss Funktion, MRE
def MRELoss(outputs, targets):
    
    loss = torch.mean(abs((outputs - targets) / targets))
    
    return loss
    

#Optimizer
learning_rate = 0.001
optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate)


#### Training

In [7]:
num_epochs = 500 #Iterationen über Datenset

train_loss = []
test_loss = []
train_accuracy = []
test_accuracy = []

#Optimierungsloop
for epoch in range(num_epochs):
    train_correct = 0
    train_total = 0
        
    for batch, (X,y) in enumerate(train_dataloader):
        
        print(X.shape)
        print(X.dtype)
        #print(X)
        
        print(X.size())
        print(y.size())
        
        net.train() #Trainingmodus
        
        # forward
        pred = net(X)  # Do the forward pass
        #loss = loss_fn(pred, y) # Calculate the loss
        loss = MRELoss(pred, y)
        
        # backward
        optimizer.zero_grad() # Clear off the gradients from any past operation
        loss.backward()       # Calculate the gradients with help of back propagation
        
        # adam step gradient descent
        optimizer.step()      # Ask the optimizer to adjust the parameters based on the gradients
        
        # Record the correct predictions for training data
        train_total += y.size(0)    
        _, predicted = torch.max(pred.data, 1)
        train_correct += (predicted == y.data).sum()

        print ('Epoch %d/%d, Iteration %d/%d, Loss: %.4f' 
               %(epoch+1, num_epochs, batch+1, len(train_dataset)//batch_size, loss.item()))

    net.eval()                 # Put the network into evaluation mode

torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 1/500, Iteration 1/25, Loss: 12.8015
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 1/500, Iteration 2/25, Loss: 29.2755
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 1/500, Iteration 3/25, Loss: 62.8378
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 1/500, Iteration 4/25, Loss: 9.1102
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 1/500, Iteration 5/25, Loss: 11.7553
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 1/500, Iteration 6/25, Loss: 3.8917
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 1/500, Iteration 7/25, Loss: 5.9343
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 1/500, Iteration 8/25, Loss: 12.9355
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 1/500, Iterati

Epoch 5/500, Iteration 1/25, Loss: 1.0870
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 5/500, Iteration 2/25, Loss: 0.9795
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 5/500, Iteration 3/25, Loss: 0.9840
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 5/500, Iteration 4/25, Loss: 0.9691
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 5/500, Iteration 5/25, Loss: 1.2205
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 5/500, Iteration 6/25, Loss: 1.0345
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 5/500, Iteration 7/25, Loss: 2.0455
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 5/500, Iteration 8/25, Loss: 1.0728
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 5/500, Iteration 9/25, Loss: 1.0539
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])


Epoch 8/500, Iteration 14/25, Loss: 1.8336
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 8/500, Iteration 15/25, Loss: 1.6837
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 8/500, Iteration 16/25, Loss: 1.2662
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 8/500, Iteration 17/25, Loss: 1.5373
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 8/500, Iteration 18/25, Loss: 1.0895
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 8/500, Iteration 19/25, Loss: 1.0584
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 8/500, Iteration 20/25, Loss: 1.2412
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 8/500, Iteration 21/25, Loss: 0.9555
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 8/500, Iteration 22/25, Loss: 1.0539
torch.Size([32, 5])
torch.float64
torch.Size(

Epoch 11/500, Iteration 12/25, Loss: 1.3673
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 11/500, Iteration 13/25, Loss: 1.7825
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 11/500, Iteration 14/25, Loss: 1.0451
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 11/500, Iteration 15/25, Loss: 3.2010
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 11/500, Iteration 16/25, Loss: 7.8583
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 11/500, Iteration 17/25, Loss: 0.9902
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 11/500, Iteration 18/25, Loss: 1.5344
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 11/500, Iteration 19/25, Loss: 2.1509
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 11/500, Iteration 20/25, Loss: 2.9401
torch.Size([32, 5])
torch.float64
to

Epoch 14/500, Iteration 12/25, Loss: 1.3380
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 14/500, Iteration 13/25, Loss: 4.0671
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 14/500, Iteration 14/25, Loss: 0.9830
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 14/500, Iteration 15/25, Loss: 1.0031
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 14/500, Iteration 16/25, Loss: 0.9166
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 14/500, Iteration 17/25, Loss: 1.4371
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 14/500, Iteration 18/25, Loss: 1.8062
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 14/500, Iteration 19/25, Loss: 2.0214
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 14/500, Iteration 20/25, Loss: 3.2529
torch.Size([32, 5])
torch.float64
to

...

Epoch 482/500, Iteration 3/25, Loss: 0.9969
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 482/500, Iteration 4/25, Loss: 0.9946
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 482/500, Iteration 5/25, Loss: 0.9414
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 482/500, Iteration 6/25, Loss: 0.9767
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 482/500, Iteration 7/25, Loss: 1.0760
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 482/500, Iteration 8/25, Loss: 0.9901
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 482/500, Iteration 9/25, Loss: 0.9963
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 482/500, Iteration 10/25, Loss: 0.9867
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 482/500, Iteration 11/25, Loss: 1.0250
torch.Size([32, 5])
torch.float64


torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 485/500, Iteration 9/25, Loss: 1.1631
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 485/500, Iteration 10/25, Loss: 1.1134
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 485/500, Iteration 11/25, Loss: 1.2021
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 485/500, Iteration 12/25, Loss: 0.9492
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 485/500, Iteration 13/25, Loss: 0.9813
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 485/500, Iteration 14/25, Loss: 0.9904
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 485/500, Iteration 15/25, Loss: 1.0265
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 485/500, Iteration 16/25, Loss: 1.0114
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Ep

Epoch 488/500, Iteration 8/25, Loss: 1.0376
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 488/500, Iteration 9/25, Loss: 1.5298
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 488/500, Iteration 10/25, Loss: 1.0709
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 488/500, Iteration 11/25, Loss: 0.9203
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 488/500, Iteration 12/25, Loss: 1.0828
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 488/500, Iteration 13/25, Loss: 0.9964
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 488/500, Iteration 14/25, Loss: 1.5577
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 488/500, Iteration 15/25, Loss: 1.0134
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 488/500, Iteration 16/25, Loss: 0.9483
torch.Size([32, 5])
torch.flo

Epoch 493/500, Iteration 20/25, Loss: 1.0983
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 493/500, Iteration 21/25, Loss: 1.4266
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 493/500, Iteration 22/25, Loss: 0.9872
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 493/500, Iteration 23/25, Loss: 3.2097
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 493/500, Iteration 24/25, Loss: 1.0928
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 493/500, Iteration 25/25, Loss: 1.2295
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 494/500, Iteration 1/25, Loss: 1.0371
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 494/500, Iteration 2/25, Loss: 0.9896
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 494/500, Iteration 3/25, Loss: 0.9854
torch.Size([32, 5])
torch.floa

torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 496/500, Iteration 19/25, Loss: 1.0195
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 496/500, Iteration 20/25, Loss: 1.0031
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 496/500, Iteration 21/25, Loss: 1.0002
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 496/500, Iteration 22/25, Loss: 1.0019
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 496/500, Iteration 23/25, Loss: 1.0004
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 496/500, Iteration 24/25, Loss: 1.1712
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 496/500, Iteration 25/25, Loss: 1.0829
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Epoch 497/500, Iteration 1/25, Loss: 1.8496
torch.Size([32, 5])
torch.float64
torch.Size([32, 5])
torch.Size([32])
Ep